In [1]:
import sqlite3
import pandas as pd
import os

class SQLiteLoader:
    def __init__(self, db_name: str, table_name: str, file_path: str):
        self.db_name = db_name
        self.table_name = table_name
        self.file_path = file_path

        db_folder = 'databases'

        # Si la carpeta no existe, la crea
        if not os.path.exists(db_folder):
            os.makedirs(db_folder)
            
        self.conn = sqlite3.connect(os.path.join(db_folder, 'data.db'))
        self.cursor = self.conn.cursor()

        self.load_data()
        self.create_table()
        self.insert_data()

    def load_data(self):
        '''Carga el dataset a dataframe'''
        self.df = pd.read_csv(self.file_path)
        print('Datos cargados desde el CSV.')

        # Intentar convertir todas las columnas a datetime si es posible
        for col in self.df.select_dtypes(include=['object']).columns:
            try:
                self.df[col] = pd.to_datetime(self.df[col],
                                              format='mixed').dt.tz_localize(None)
            except Exception:
                pass  # Si la conversión falla, la dejamos como está

        print('Columnas datetime detectadas automáticamente.')

    def create_table(self):
        '''Crea la tabla si no existe, detectando los tipos de datos desde el DataFrame'''
        columns_types = ['id INTEGER PRIMARY KEY AUTOINCREMENT']
        for col, dtype in self.df.dtypes.items():
            if 'int' in str(dtype):
                col_type = 'INTEGER'
            elif 'float' in str(dtype):
                col_type = 'FLOAT'
            elif 'datetime' in str(dtype):
                col_type = 'DATETIME'
            else:
                col_type = 'VARCHAR'
            columns_types.append(f'{col} {col_type}')
        create_query = f'''CREATE TABLE IF NOT EXISTS {self.table_name}
                            ({', '.join(columns_types)})'''
        
        self.cursor.execute(create_query)
        self.conn.commit()
        print('Tabla creada o ya existente.')

    def insert_data(self):
        '''Inserta los datos en la tabla'''

        # Convertir datetime a string ISO 8601 antes de insertar
        for col in self.df.columns:
            if self.df[col].dtype == "datetime64[ns]":
                self.df[col] = self.df[col].dt.strftime('%Y-%m-%d %H:%M:%S')
        
        cols_without_id = [col for col in self.df.columns if col != 'id']
        placeholders = ', '.join(['?' for _ in cols_without_id])
        
        insert_query = f'''
        INSERT INTO {self.table_name}
        ({", ".join(cols_without_id)})
        VALUES ({placeholders})
        '''
        self.cursor.executemany(insert_query, self.df.values.tolist())
        self.conn.commit()
        print('Datos insertados en la base de datos.')

    def close_connection(self):
        '''Cierra la conexión con la base de datos'''
        self.conn.close()
        print('Conexión cerrada.')

In [2]:
db_loader = SQLiteLoader(db_name='data.db',
                         table_name='shared_car_locations',
                         file_path='../datasets/autotel-shared-car-locations/2020_02_25.csv')

Datos cargados desde el CSV.
Columnas datetime detectadas automáticamente.
Tabla creada o ya existente.
Datos insertados en la base de datos.


In [ ]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, Float, String, DateTime, MetaData, Table
import os

class SQL_Loader:
    def __init__(self, db_name: str, table_name: str, file_path: str):
        self.db_name = db_name
        self.table_name = table_name
        self.file_path = file_path

        db_folder = 'databases'

        # Si la carpeta no existe, la crea
        if not os.path.exists(db_folder):
            os.makedirs(db_folder)
        
        db_path = os.path.join(db_folder, 'data.db')
        
        self.engine = create_engine(f"sqlite:///{db_path}")
        self.metadata = MetaData()
        self.df = None

        self.load_data()
        self.create_table()
        self.insert_data()
        self.close_connection()

    def load_data(self):
        """Carga el dataset"""
        self.df = pd.read_csv(self.file_path)
        print("Datos cargados desde el CSV.")

        # Intentar convertir todas las columnas a datetime si es posible
        for col in self.df.select_dtypes(include=['object']).columns:
            try:
                self.df[col] = pd.to_datetime(self.df[col],
                                              format='mixed').dt.tz_localize(None)
            except Exception:
                pass  # Si la conversión falla, la dejamos como está

        print("Columnas datetime detectadas automáticamente.")

    def create_table(self):
        """Crea la tabla con un índice autoincremental si no existe"""
        columns = [
            Column("id", Integer, primary_key=True, autoincrement=True)  # ID autoincremental
        ]

        # Inferir tipos de datos de Pandas a SQLAlchemy
        for col, dtype in self.df.dtypes.items():
            if "int" in str(dtype):
                col_type = Integer
            elif "float" in str(dtype):
                col_type = Float
            elif "datetime" in str(dtype):
                col_type = DateTime
            else:
                col_type = String  # Cualquier otro tipo lo tratamos como texto
            
            columns.append(Column(col, col_type))

        # Definir la tabla
        Table(self.table_name, self.metadata, *columns)

        # Crear la tabla si no existe
        self.metadata.create_all(self.engine)
        print(f"Tabla '{self.table_name}' creada o ya existente.")

    def insert_data(self):
        """Inserta los datos en la tabla sin sobrescribir su estructura"""
        # Convertir datetime a string ISO 8601 antes de insertar
        for col in self.df.columns:
            if self.df[col].dtype == "datetime64[ns]":
                self.df[col] = self.df[col].dt.strftime('%Y-%m-%d %H:%M:%S')
        
        with self.engine.connect() as conn:
            self.df.to_sql(self.table_name, conn, if_exists='append', index=False)
        print("Datos insertados en la base de datos.")

    def close_connection(self):
        """SQLAlchemy maneja las conexiones automáticamente"""
        print("Conexión gestionada por SQLAlchemy.")

In [2]:
db_loader = SQL_Loader(db_name='data.db',
                       table_name='shared_car_locations_v2',
                       file_path='../datasets/autotel-shared-car-locations/2020_02_25.csv')

Datos cargados desde el CSV.
Columnas datetime detectadas automáticamente.
Tabla 'shared_car_locations_v2' creada o ya existente.
Datos insertados en la base de datos.
Conexión gestionada por SQLAlchemy.
